In [1]:
from keras import models, layers, Input

input_tensor = Input(shape=(64,))

x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = models.Model(input_tensor, output_tensor)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [2]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [3]:
import numpy as np

train_x = np.random.random((1000, 64))
train_y = np.random.random((1000, 10))

model.fit(train_x, train_y, epochs=10, batch_size=128)
score = model.evaluate(train_x, train_y)

Epoch 1/10
8/8 [==============================] - 1s 3ms/step - loss: 11.7150 - acc: 0.0989
Epoch 2/10
8/8 [==============================] - 0s 5ms/step - loss: 11.9371 - acc: 0.1024
Epoch 3/10
8/8 [==============================] - 0s 5ms/step - loss: 12.7812 - acc: 0.1028
Epoch 4/10
8/8 [==============================] - 0s 7ms/step - loss: 13.8088 - acc: 0.1167
Epoch 5/10
8/8 [==============================] - 0s 4ms/step - loss: 15.1015 - acc: 0.1080
Epoch 6/10
8/8 [==============================] - 0s 4ms/step - loss: 16.9867 - acc: 0.1120
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 19.1268 - acc: 0.1160
Epoch 8/10
8/8 [==============================] - 0s 3ms/step - loss: 21.3023 - acc: 0.1080
Epoch 9/10
8/8 [==============================] - 0s 3ms/step - loss: 23.9927 - acc: 0.1137
Epoch 10/10
32/32 [==============================] - 0s 2ms/step - loss: 31.6086 - acc: 0.1040


In [4]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(0, 1, size=(num_samples, answer_vocabulary_size))

model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 7s 127ms/step - loss: 0.0000e+00 - acc: 0.0025
Epoch 2/10
8/8 [==============================] - 2s 192ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 1s 103ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 1s 119ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 5/10
8/8 [==============================] - 1s 155ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 6/10
8/8 [==============================] - 1s 119ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 7/10
8/8 [==============================] - 1s 108ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 8/10
8/8 [==============================] - 1s 122ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 9/10
8/8 [==============================] - 1s 144ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 10/10
8/8 [==============================] - 1s 127ms/step - loss: 0.0000e+00 - acc: 0.00

In [5]:
vocabulary_size = 50000
num_income_group = 10

posts_input = Input(shape=(None,), dtype='int32', name='post')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age = layers.Dense(1, name='age')(x)
income = layers.Dense(num_income_group, activation='softmax', name='income')(x)
gender = layers.Dense(1, name='gender', activation='sigmoid')(x)
model = models.Model(posts_input, [age, income, gender])

In [6]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'], loss_weights=[0.25, 1., 10.])

In [7]:
# from keras import layers
# branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)
# branch_b = layers.Conv2D(128, 1, activation='relu')(x)
# branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)
# branch_c = layers.AveragePooling2D(3, strides=2)(x)
# branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)
# branch_d = layers.Conv2D(128, 1, activation='relu')(x)
# branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
# branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)
# output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

In [8]:
lstm = layers.LSTM(32)

left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
prediction = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], prediction)

In [10]:
from keras import applications
xception_base = applications.Xception(weights=None, include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features])